<div style='background-image: url("../../share/images/header.svg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Computational Seismology</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)"> SBP-SAT finite difference method for the 2D elastic wave equation in velocity-stress form </div>
        </div>
    </div>
</div>


---

This notebook is based on the paper [Dynamic earthquake rupture simulations on nonplanar faults embedded in 3D geometrically complex, heterogeneous Earth models](https://pangea.stanford.edu/~edunham/publications/Duru_Dunham_FD3d_JCP16.pdf)


##### Authors:
* Kenneth Duru

## Description

### Background 

This notebook simulates  dynamic earthquake ruptures and elastic wave propagation in two space dimensions (2D). Spontaneously propagating shear ruptures on a frictional interface in an elastic solid is a useful idealization of natural earthquakes. The conditions relating slip-rate and fault strength are often expressed as nonlinear friction laws.

During an earthquake, two sides of the fault, initially held in contact by a high level of frictional resistance, slip suddenly when that resistance catastrophically decreases, generating strong ground shaking which is carried by seismic waves to remote areas, far away from fault zones.

This notebook describes and implements a high order accurate finite difference method for enforcing nonlinear friction laws in a consistent and provably stable manner, suitable for efficient explicit time integration and accurate propagation of seismic waves in heterogeneous media with free surface boundary conditions.

<br/><br/>

### The elastic wave equation
--------------------------------------------------------------------
Consider two elastic solids separated by a fault at the interface $x = 0$. In each elastic solid, wave motion is described by the elastic wave equation, in velocity-stress formulation: 
<br/><br/> 
\begin{align}
{\rho} {\partial_t v_i} = {\frac{ \partial \sigma_{ij}}{\partial x_j},  \quad}  {\quad}   S_{klij}{\partial_t \sigma_{ij}} = \frac{1}{2}  \cdot (\frac{\partial v_k}{\partial x_l} + \frac{\partial v_l}{\partial x_k}), \quad i, j = 1, 2,
\end{align}
<br/> <br/>
 with particle velocities ${v_i}$, stress tensor ${\sigma_{ij} }$, compliance tensor $S_{klij}$, and density ${\rho}$.
<br/><br/>

-----------------------------------------------------------------

On the fault there is the velocity vector $\left(v_{n}, v_{m}\right)=\left(v_{x}, v_{y}\right)$, and the traction vector $\left(T_{n}, T_{m}\right)=\left(\sigma_{xx}, \sigma_{xy}\right)$, where $v_{n}$, $T_{n}$ are the normal components and $v_{m}$, $T_{m}$ being the shear components on the  interface. Denoting the fields in the positive and the negative parts of the fault with the superscripts $\pm$, we introduce the jumps in particle velocities by $\lbrack{v_j}\rbrack = {v_j}^{+}-{v_j}^{-}$, with $j = m, n$, and  the total traction by $T_j = T_{0j} + \Delta{T_j}$, where $T_{0j}$ are the initial background tractions and $\Delta{T_j}$ are the traction changes on the fault. The conditions on the fault connecting the two elastic solids are
<br/><br/> 
\begin{align}
\text{force balance}&: \quad T_{j}^{+} = T_{j}^{-} = T_{j}, \quad j = m, n, \\
\text{no opening}&: \quad \lbrack{v_n}\rbrack = 0,\\
\text{friction laws}&: \quad T_m = \sigma_n\frac{f(V, \theta)}{V}\lbrack{v_m}\rbrack, \quad V = \left|\lbrack{v_m}\rbrack\right|.
\end{align}
<br/> <br/>
 Here $\sigma_n > 0$ is the compressive normal stress and $ f(V, \theta) \ge 0 $ is the nonlinear fricition coefficient relating to the fault's shear strength, $\tau = |T_m| > 0$, and will be described  in more detail below.
<br/><br/>

-------------------------------------------------------------

### Fracture modes 

There are three modes of fracture as illustrated below. Mode I fracture is an in plane motion where the direction of movement is perpendicular to the fracture itself, this opening can be described by a single component tensile component. Mode II is an in plane fracture that moves parallel to the fracture, it can be described by a compressive component that is perpendicular to the fracture (non-opening) and another parallel component that is creating shearing forces. This is a widely used mode for rupture dynamics because, both P and S waves emanate from this mode. This can allow 'super shear' ruptures, shear ruptures propagating above the S wave velocity. Mode III is an out of plane shearing motion and can be described in one out of plane component. This mode of rupture has a strict upper bound of the S wave velocity. Since earthquakes don't open wide chasms into the earth, mode I is not used here. <br/>


<p style="width:75%;float:right;padding-left:50px">
<img src=fracmodes.png>
<span style="font-size:smaller">
</span>
</p>
<br/>
Figure source: https://www.doitpoms.ac.uk/tlplib/brittle_fracture/same.php

### Friction laws

It is necessary to consider the role of friction in all of this because it modulates the stresses and slip speed as the separate sides of the fault slip past each other. The first thing to consider is that as the fault slips it experiences shear strength ${\tau}$ which is exerted by the normal stresses on the fault ${\sigma_n}$. The way these two variables are related is through the friction coefficient ${f}$: ${\quad} {\tau} = f \cdot {\sigma_n}$. 

 For the purposes of this notebook  we consider two widely used models of friction, *slip-weakening* and *rate-and-state*. <br/>


<p style="width:75%;float:right;padding-left:50px">
<img src=RS_graph.png>
<span style="font-size:smaller">
</span>
</p>
#### Slip Weakening Friction
Slip weakening friction describes a friction behavior in which friction coefficient weakens linearly with slip.  When the fault is at rest the friction coefficient is denoted by static friction coefficient ${f_s}$, when the fault is in motion the friction coefficient is denoted by the dynamic friction coefficient ${f_d}$. An earthquake begins when the load $\tau$ at a finite patch on the fault overcomes the peak frictional strength, $\tau \ge \tau_p$, ${\quad \tau_p = f_s \cdot \sigma_n}$.  As the earthquake continues the fault surfaces slide relative to each other  and the frictional coefficient evolves from linearly from static friction coefficient $f_s$  to the dynamic friction coefficient ${f_d}$. The fault is fully weakened when the slip reaches the critical slip ${D_c}$. This can be seen in the figure above which plots the friction coefficient as a function of slip.
 
#### Rate and State Friction

Rate and state friction is a constitutive law that was empirically found and is considered more realistic than slip weakening. It takes the form:  

\begin{align}
 &\tau = \sigma_n \cdot f(V, \theta), \quad f(V, \theta) = [ f_0 + a \cdot ln(\frac {V}{V_0}) + b \cdot ln( \frac{V_0 \theta}{D_c})], \\
  &\frac {d \theta}{dt} = G(V, \theta).
 \end{align}

In this equation ${V}$ is the current slip velocity and ${V_0}$ is a reference slip velocity while ${f_0}$ is the steady state friction coefficient at the reference velocity: ${f_{V= V_0}}$. There is the state variable ${\theta}$, and the state evolution law $G(V, \theta)$. We consider specifically the aging law: ${ G(V, \theta) = 1 - \frac {V\theta }{D_c}.\quad}$ At steady state the state variable is proportional to the critical distance divided by velocity ${ \theta_{ss} = \frac {D_c}{V}}$  <br/>

Here ${a}$ is the *direct effect* and is used to model how the system responds to velocity changes. ${b}$ is the *evolution effect* and describes the magnitude of the steady state friction.  ${D_c}$ is the critical distance and corresponds to the slip length over which the evolution of ${a \to b}$ occurs. The relationship of ${a}$ and ${b}$ to the slip weakening friction variables is as follows: ${\quad b = \frac {df_s}{d(ln(t))} \quad, \quad (a-b) = \frac {df_d}{d(ln(V))} \quad}$ where ${t}$ is the amount of time of contact without an earthquake. It is interesting to notice that there is a relationship between slip velocity ${V}$ and dynamic friction ${f_d}$ that depends on the difference ${(a-b)}$. If ${(a-b)>0}$ it shows that there is a velocity strengthening relationship ${\frac {df_d}{d(ln(V))} > 0}$. Any rupture that enters this region will arrest and is called a *stable* region. If ${(a-b)<0}$ it shows that there is a velocity weakening relationship ${\frac {df_d}{d(ln(V))} < 0}$, this is where earthquakes can nucleate and grow and is called *unstable*. On the border between *stable* and *unstable* regions is a zone of *conditional stability* where ruptures can sustain propagation while not growing or terminating. This notebook only exists in the unstable zone.



The equations are discretized using the SBP-SAT finite difference scheme. Time integration is performed using the classical fourth order accurate Runge-Kutta method. For more elaborate discussions, we refer the reader to the references and the  notebooks on the SBP-SAT method. A summary of the numerical implementations used in the this notebook is presentated in the flow-chart below.

<img src=2Dnotebook_flowchart.jpg>

### Putting it together

In the parameter window you can modulate the material properties, run time, domain size, and CFL criterion. In addition, you can choose rate and state friction or slip weakening friction, notice the differences that occur with this choice. In the Calculations and plotting cell the resulting on fault tractions and slip velocities are solved for in the RK4_2D.elastic_RK4_2D function, which also calls on the other associated functions. The bottom cell contains 5 output plots of the rupture behavior. 



The plot that is observed immediately under this cell is the particle velocity plotted in the entire domain. The fault runs vertically through the center of the model and is marked by a vertical line. When this animation finishes the seismograms of the earthquake pop up under this. In the last cell there are three plots that come from the data generated in the previous cell. These three plots show the on fault slip, slip rate, and tractions as they evlove through time. Lastly there is a plot that shows the slip rate on the fault through time.
 

#### References 
Byerlee, J. “Friction of Rocks.” Rock Friction and Earthquake Prediction, vol. 116, 1978, pp. 615–626., doi:10.1007/978-3-0348-7182-2_4. <br/>
Duru, Kenneth, and Eric M. Dunham. “Dynamic Earthquake Rupture Simulations on Nonplanar Faults Embedded in 3D 
Geometrically Complex, <br/>${\quad}$ Heterogeneous Elastic Solids.” Journal of Computational Physics, vol. 305, 2016, pp. 185–207., doi: 10.1016/j.jcp.2015.10.0215<br/>
Gustafsson, Bertil. High Order Difference Methods for Time Dependent PDE. Springer, 2008.<br/>
Kozdon, J. E., E. M. Dunham, and J. Nordström (2012), Interaction of waves with frictional interfaces using summation-by-parts difference operators:<br/>${\quad}$  Weak enforcement of nonlinear boundary conditions, Journal of Scientific Computing, 50(2), 341-367, doi:10.1007/s10915-011-9485-3<br/>
Leeman, John. “Modeling Rate and State Friction with Python.” SciPy 2016. Austin, USA.<br/>
Scholz, Christopher H. “Earthquakes and Friction Laws.” Nature, vol. 391, no. 6662, 1998, pp. 37–42., doi:10.1038/34097.

**** Exercises****



In [12]:
# Parameters initialization and plotting the simulation
# Import necessary routines
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

# Reload modules to pick up any changes
import importlib
import sys

# Remove cached modules
modules_to_reload = ['RK4_2D', 'rate2d', 'first_derivative_sbp_operators', 
                     'boundarycondition', 'interface', 'interfacedata']
for mod in modules_to_reload:
    if mod in sys.modules:
        del sys.modules[mod]

# Now import RK4_2D
import RK4_2D

#plt.switch_backend("TkAgg")          # plots in external window
plt.switch_backend("nbagg")           # plots within this notebook

In [13]:

Lx = 10.0     # length of the domain (x-axis)
Ly = 20.0     # width of the domain (y-axis)
tend = 10.0

nx = 26       # grid points in x
ny = 51       # grid points in y
CFL = 0.5


dx = Lx/nx    # spatial step in x
dy = Ly/ny    # spatial step in y

isnap = 5     # snapshot frequency

nf = 5        # number of fields
cs = 3.464    # shear wave speed
cp = 6.0      # compresional wave speed
rho = 2.6702  # density

# source parameters
x0 = -15.0        # [km]
y0 = 7.5         # [km]
t0 = 0.0         # [s]
T =  0.1         # [s]
M0 = 00.0      # [MPa]

source_type = 'Gaussian' # 'Gaussian', 'Brune'

# RS: rate-and-state friction law
# SW: slip-weakening friction law
fric_law = 'RS'   #RS OR SW

# fracture mode
# Mode II crack: In-plane shear
# Mode III crack: out-of-plane (anti-plane) shear
mode = 'II'   #  'II' or 'III'

# Mode II crack: In-plane shear
if mode == 'II':
    nf = 5        # number of fields [vx, vy, sxx, syy, sxy] 

# Mode III crack: out-of-plane (anti-plane) shear    
if mode == 'III':
    nf = 3        # number of fields  [vz, sxz, sxy]  


# RS - add correct parameters

M = [0, 0, 1., 1., 0]

source_parameter = [x0, y0, t0, T, M0, source_type, M]

# extract Lame parameters
mu = rho*cs**2
Lambda = rho*cp**2-2.0*mu

dt = CFL/np.sqrt(cp**2 + cs**2)*dx    # Time step
nt = int(np.round(tend/dt))                             # number of time steps

order = 4        # spatial order of accuracy

# Model type, available are "homogeneous", "fault_zone",
# "surface_low_velocity_zone", "random", "topography",
# "slab"
model_type = "homogeneous"

# Initialize velocity model
Mat_l = np.zeros((nx, ny, 3))

Mat_r = np.zeros((nx, ny, 3))

if model_type == "homogeneous":
    Mat_l[:,:,0] += rho
    Mat_l[:,:,1] += Lambda
    Mat_l[:,:,2] += mu
    
    Mat_r[:,:,0] += rho
    Mat_r[:,:,1] += Lambda
    Mat_r[:,:,2] += mu

elif model_type == "random":
    pert_l = 0.4
    r_rho_l = 2.0 * (np.random.rand(nx, ny) - 0.5) * pert_l
    r_mu_l = 2.0 * (np.random.rand(nx, ny) - 0.5) * pert_l
    r_lambda_l = 2.0 * (np.random.rand(nx, ny) - 0.5) * pert_l    
    Mat_l[:,:,0] += rho*(1.0 + r_rho_l)
    Mat_l[:,:,1] += Lambda*(1.0 + r_lambda_l)
    Mat_l[:,:,2] += mu*(1.0 + r_mu_l)
    
    pert_r = 0.4
    r_rho_r = 2.0 * (np.random.rand(nx, ny) - 0.5) * pert_r
    r_mu_r = 2.0 * (np.random.rand(nx, ny) - 0.5) * pert_r
    r_rambda_r = 2.0 * (np.random.rand(nx, ny) - 0.5) * pert_r
    Mat_r[:,:,0] += rho*(1.0 + r_rho_r)
    Mat_r[:,:,1] += Lambda*(1.0 + r_rambda_r)
    Mat_r[:,:,2] += mu*(1.0 + r_mu_r)
    
# Initialize pressure at different time steps and the second
# derivatives in each direction
F_l = np.zeros((nx, ny, nf))
Fnew_l = np.zeros((nx, ny, nf))
X_l = np.zeros((nx, ny))
Y_l = np.zeros((nx, ny))
p_l = np.zeros((nx, ny))

F_r = np.zeros((nx, ny, nf))
Fnew_r = np.zeros((nx, ny, nf))
X_r = np.zeros((nx, ny))
Y_r = np.zeros((nx, ny))
p_r = np.zeros((nx, ny))




for i in range(0, nx):
    for j in range(0, ny):
        X_l[i,j] = -Lx + i*dx
        Y_l[i,j] = j*dy
        
        X_r[i,j] = i*dx
        Y_r[i,j] = j*dy
        
        #F_l[i,j,1] = -10**-12 
        #F_r[i,j,1] =  10**-12
        

Y_fault =np.zeros((ny, 1)) 
Y0 = 10

for j in range(0, ny):
        #Y_fault[j, 0] = j*dy
        
        if np.abs(j*dy-Y0) <= 1.5:
            Y_fault[j, 0] = 1.0

# FRICTION PARAMETERS

# Calculations and plottings for on fault slip velocity and stress


slip = np.zeros((ny, 1))
psi = np.zeros((ny, 1))
FaultOutput = np.zeros((ny, nt, 6)) # c
FaultOutput0 = np.zeros((ny, 6))
DomainOutput_l = np.zeros((nx, ny, nt, nf)) #
DomainOutput_r = np.zeros((nx, ny, nt, nf))

if fric_law not in ('SW', 'RS'):
    # Choose friction law: fric_law
    # We use linear (LN: T = alpha*v, alpha >=0)
    # Slip-weakening (SW)
    # Rate-and-state friction law (RS)
    
     print('friction law not implemented. choose fric_law = SW or fric_law = RS')
     exit(-1)
    
if fric_law  in ('SW'):
     
     slip = np.zeros((ny, 1))                        # initial slip (in m)
     slip_new = np.zeros((ny, 1))
     Tau_0 = np.ones((ny, 1))*(70+11.6*Y_fault)   
     alp_s = np.ones((ny, 1))*0.677                          # stastic friction
     alp_d = np.ones((ny, 1))*0.525                          # dynamic friction
     D_c = np.ones((ny, 1))*0.4                              # critical slip
     sigma_n = -np.ones((ny, 1))*120.0                        # normal stress 
     
        
     # These are not needed for the slip weakening case   
     psi = np.ones((ny, 1))*0.0                         # initial condition for the state variable in friction law
     psi_new = np.ones((ny, 1))*0.0
     L0 = np.ones((ny, 1))*1.0                               # state evolution distance
     f0 = np.ones((ny, 1))*1.0                               # referance friction coeff
     a = np.ones((ny, 1))*1.0                                # direct effect 
     b = np.ones((ny, 1))*1.0                                # evolution parameter 
     V0 = np.ones((ny, 1))*1.0                               # reference slip rate
     alpha = np.ones((ny, 1))*1e1000000                      # initial friction coefficient
        
     FaultOutput[:, 0, 2] = sigma_n[:,0]   
     FaultOutput[:, 0, 3] = Tau_0[:,0] 
     FaultOutput[:, 0, 4] = slip[:,0] 
     FaultOutput[:, 0, 5] = psi[:,0]   
    
if fric_law  in ('RS'):
     alpha = np.ones((ny, 1))*1e1000000                      # initial friction coefficient                                                                                   
     slip = np.ones((ny, 1))*0.0                         # initial slip (in m) 
     slip_new = np.zeros((ny, 1))
     #Tau_0 = np.ones((ny, 1))*81.24+0.1*0.36                 # initial load (81.24 in MPa), slight increase will unlock the fault   
     psi = np.ones((ny, 1))*0.4367                      # initial condition for the state variable in friction law
     psi_new = np.ones((ny, 1))*0.0
     L0 = np.ones((ny, 1))*0.02                              # state evolution distance
     f0 = np.ones((ny, 1))*0.6                               # referance friction coeff
     a = np.ones((ny, 1))*0.008                              # direct effect 
     b = np.ones((ny, 1))*0.012                              # evolution parameter 
     V0 = np.ones((ny, 1))*1.0e-6                            # reference slip rate
     sigma_n = -np.ones((ny, 1))*120.0                        # background normal stress 
     Tau_0 = np.ones((ny, 1))*75 #-2*0.2429*sigma_n*Y_fault
     Vin = np.ones((ny, 1))*2.0e-12 
     theta = L0/V0*np.exp(((a*np.log(2.0*np.sinh(75/(a*120)))-f0-a*np.log(Vin/V0))/b))
     psi[:,0] = f0[:,0] + b[:,0]*np.log(V0[:,0]/L0[:,0]*theta[:,0])
                          
    
     # These are not needed for the rate and state case   
     alp_s = np.ones((ny, 1))*1.0                             # stastic friction
     alp_d = np.ones((ny, 1))*1.0                             # dynamic friction
     D_c = np.ones((ny, 1))*1.0                               # critical slip
        
     FaultOutput[:, 0, 2] = sigma_n[:,0]  
     FaultOutput[:, 0, 3] = Tau_0[:,0]
     FaultOutput[:, 0, 4] = slip[:,0]
     FaultOutput[:, 0, 5] = psi[:,0] 
        
friction_parameters = np.zeros((12, ny))    
#friction_parameters = [alpha, alpha, Tau_0, L0, f0, a, b, V0, sigma_n, alp_s, alp_d, D_c]    
#                        0         1      2     3   4  5  6   7    8       9      10  11

for j in range(0, ny):
        friction_parameters[0, j] = alpha[j, 0]
        friction_parameters[1, j] = alpha[j, 0]
        friction_parameters[2, j] = Tau_0[j, 0]
        friction_parameters[3, j] = L0[j, 0]
        friction_parameters[4, j] = f0[j, 0]
        friction_parameters[5, j] = a[j, 0]
        
        
        friction_parameters[6, j] = b[j, 0]
        friction_parameters[7, j] = V0[j, 0]
        friction_parameters[8, j] = sigma_n[j, 0]
        friction_parameters[9, j] = alp_s[j, 0]
        friction_parameters[10, j] = alp_d[j, 0]
        friction_parameters[11, j] = D_c[j, 0]
        
        #if np.abs(j*dy-Y0) <= 1.5:
        #    Y_fault[j, 0] = 1.0
            
############ END FRICTION PARAMETERS




# Receiver locations left
rx_l = np.array([-3])
ry_l = np.array([0])

irx_l = np.array([1])
iry_l = np.array([0])

for i in range(len(rx_l)):
    irx_l[i] = (np.ceil(rx_l[i]/dx))+(nx-1)
    iry_l[i] = (np.ceil(ry_l[i]/dy))

seisvx_l = np.zeros((len(irx_l), nt))
seisvy_l = np.zeros((len(irx_l), nt))

# Receiver locations right

rx_r = np.array([3])
ry_r = np.array([1])

irx_r = np.array([1])
iry_r = np.array([0])

for i in range(len(rx_r)):
    irx_r[i] = (np.ceil(rx_r[i]/dx))
    iry_r[i] = (np.ceil(ry_r[i]/dy))

seisvx_r = np.zeros((len(irx_r), nt))
seisvy_r = np.zeros((len(irx_r), nt))



# Boundary reflection coefficients: 0<= r[j] <= 1
r_l = np.array([0.,0.,1.,0.])
r_r = np.array([0.,0.,1.,0.])

# required for seismograms
ir_l = np.arange(len(irx_l))
ir_r = np.arange(len(irx_r))


print(irx_l)
print(iry_l)

#exit()



#################################################

# Time-stepping 
for it in range(nt):
    
    t = it*dt
    #4th order Runge-Kutta 
    RK4_2D.elastic_RK4_2D(Fnew_l, F_l, Mat_l, X_l, Y_l, t, nf, nx, ny, dx, dy, dt, order, r_l, source_parameter, Fnew_r, F_r, Mat_r, X_r, Y_r, r_r, friction_parameters, slip,  psi, slip_new, psi_new, fric_law, FaultOutput0, Y0)

    #(DF_l, F_l, Mat_l, X_l, Y_l, t, nf, nx, ny, dx, dy, dt, order, r_l, source_parameter, DF_r, F_r, Mat_r, X_r, Y_r, r_r,friction_parameters, slip, psi, DF_slip, DF_psi, fric_law, FaultOutput0, Y0):
    # update fields
    F_l = Fnew_l
    F_r = Fnew_r
    slip = slip_new 
    psi = psi_new
    
    FaultOutput[:, it, :] = FaultOutput0
    DomainOutput_l[:, :, it, :] = F_l
    DomainOutput_r[:, :, it, :] = F_r

    
    #extract particle velocity for visualization
    #p_l = np.sqrt(F_l[:,:,0]**2 + F_l[:,:,1]**2)
    #p_r = np.sqrt(F_r[:,:,0]**2 + F_r[:,:,1]**2)
    
    # update time
    t = it*dt
    
    if it % isnap == 0:
        
        print('time =', t, 'iteration count =', it)
    
    p_l = F_l[:,:,1]
    p_r = F_r[:,:,1]
    
    

    
    if mode == 'II':
        # Save seismograms
        seisvx_l[ir_l, it] = F_l[irx_l[ir_l], iry_l[ir_l], 0]
        seisvy_l[ir_l, it] = F_l[irx_l[ir_l], iry_l[ir_l], 1]
    
        seisvx_r[ir_r, it] = F_r[irx_r[ir_r], iry_r[ir_r], 0]
        seisvy_r[ir_r, it] = F_r[irx_r[ir_r], iry_r[ir_r], 1]
        

    if mode == 'III':
        # Save seismograms
        seisvx_l[ir_l, it] = F_l[irx_l[ir_l], iry_l[ir_l], 0]
        seisvx_r[ir_r, it] = F_r[irx_r[ir_r], iry_r[ir_r], 0]
        
    


[18]
[0]
time = 0.0 iteration count = 0
time = 0.1387871401774455 iteration count = 5
time = 0.277574280354891 iteration count = 10
time = 0.41636142053233655 iteration count = 15
time = 0.555148560709782 iteration count = 20
time = 0.6939357008872276 iteration count = 25
time = 0.8327228410646731 iteration count = 30
time = 0.9715099812421185 iteration count = 35
time = 1.110297121419564 iteration count = 40
time = 1.2490842615970097 iteration count = 45
time = 1.3878714017744551 iteration count = 50
time = 1.5266585419519005 iteration count = 55
time = 1.6654456821293462 iteration count = 60
time = 1.8042328223067916 iteration count = 65
time = 1.943019962484237 iteration count = 70
time = 2.0818071026616827 iteration count = 75
time = 2.220594242839128 iteration count = 80
time = 2.3593813830165735 iteration count = 85
time = 2.4981685231940194 iteration count = 90
time = 2.636955663371465 iteration count = 95
time = 2.7757428035489102 iteration count = 100
time = 2.9145299437263557

In [23]:
from matplotlib.animation import FuncAnimation, PillowWriter

plt.ioff()

# Create animation figure
fig_anim = plt.figure(figsize=(8, 6))
ax_anim = fig_anim.add_subplot(111)

p_l = DomainOutput_l[:, :, 0, 1]
p_r = DomainOutput_r[:, :, 0, 1]   

v = 2.0
image_r = ax_anim.imshow(np.squeeze(np.append([p_l.transpose()],[p_r.transpose()],axis=2)), 
                          aspect='auto', extent=[-Lx,Lx,Ly,0],
                          cmap='seismic', vmin=-v, vmax=+v, 
                          interpolation='none')

# Plot the receivers
for x, y in zip(rx_l, ry_l):
    ax_anim.text(x, y, '+')
    
for x, y in zip(rx_r, ry_r):
    ax_anim.text(x, y, '+')

ax_anim.text(x0, y0, 'o')
plt.colorbar(image_r, ax=ax_anim)
ax_anim.set_xlabel('x')
ax_anim.set_ylabel('y')

# Animation update function
def update_frame(frame_it):
    if mode == 'II':
        p_l = DomainOutput_l[:, :, frame_it, 1]
        p_r = DomainOutput_r[:, :, frame_it, 1]
    elif mode == 'III':
        p_l = DomainOutput_l[:, :, frame_it, 0]
        p_r = DomainOutput_r[:, :, frame_it, 0]    
    
    p_b = np.squeeze(np.append([p_l.transpose()],[p_r.transpose()],axis=2))
    image_r.set_data(p_b)
    ax_anim.set_title(f"time: {frame_it*dt:.2f} s")
    return [image_r]

# Create animation (sample every 10th frame to reduce file size)
frames = range(0, nt-1, 10)
anim = FuncAnimation(fig_anim, update_frame, frames=frames, blit=True, interval=50)

# Save animation as GIF
print("Saving wave animation as GIF...")
writer = PillowWriter(fps=20)
anim.save('wave_animation.gif', writer=writer)
print("Saved: wave_animation.gif")

# Delete animation to free memory
del anim

fig4 = plt.figure(figsize=(10, 4))
if mode == 'II':
    ay1 = fig4.add_subplot(2,2,1)
    ymax = seisvx_l.ravel().max()
    time = np.arange(nt) * dt
    for ir_l in range(len(seisvx_l)):
        line14 = ay1.plot(time, seisvx_l[ir_l, :] + ymax * ir_l)
        #plt.xlabel('Time (s)')
        plt.ylabel('vx (m/s)')
        ay1.set_xlim([0, tend])
        ay1.set_ylim([-3, 3])

    ay2 = fig4.add_subplot(2,2,2)
    ymax = seisvy_l.ravel().max()
    for ir_l in range(len(seisvy_l)):
        line24 = ay2.plot(time, seisvy_l[ir_l, :] + ymax * ir_l)
        #plt.xlabel('Time (s)')
        plt.ylabel('vy (m/s)')
        ay2.set_xlim([0, tend])
        ay2.set_ylim([-3, 3])


    
    ay3 = fig4.add_subplot(2,2,3)
    ymax = seisvx_r.ravel().max()
    time = np.arange(nt) * dt
    for ir_r in range(len(seisvx_r)):
        line34 = ay3.plot(time, seisvx_r[ir_r, :] + ymax * ir_r)
        plt.xlabel('Time (s)')
        plt.ylabel('vx (m/s)')
        ay3.set_xlim([0, tend])
        ay3.set_ylim([-3, 3])

    ay4 = fig4.add_subplot(2,2,4)
    ymax = seisvy_r.ravel().max()
    for ir_r in range(len(seisvy_r)):
        line44 = ay4.plot(time, seisvy_r[ir_r, :] + ymax * ir_r)
        plt.xlabel('Time (s)')
        plt.ylabel('vy (m/s)')
        ay4.set_xlim([0, tend])
        ay4.set_ylim([-3, 3])
        
    
if mode == 'III':
    plt.subplot(2,1,1)
    ymax = seisvx_l.ravel().max()
    time = np.arange(nt) * dt
    for ir_l in range(len(seisvx_l)):
        plt.plot(time, seisvx_l[ir_l, :] + ymax * ir_l)
        plt.xlabel('Time (s)')
        plt.ylabel('vx (m/s)')
        #set_xlim((0, tend))
        #set_ylim((-3, 3))

       
    plt.subplot(2,1,2)
    ymax = seisvx_r.ravel().max()
    time = np.arange(nt) * dt
    for ir_r in range(len(seisvx_r)):
        plt.plot(time, seisvx_r[ir_r, :] + ymax * ir_r)
        plt.xlabel('Time (s)')
        plt.ylabel('vx (m/s)')
        #set_xlim((0, tend))
        #set_ylim((-3, 3))

# Save seismogram figure
fig4.tight_layout()
fig4.savefig('seismograms.png', dpi=300, bbox_inches='tight')
print("Saved: seismograms.png")
plt.show()
        

Saving wave animation as GIF...
Saved: wave_animation.gif
Saved: seismograms.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
from matplotlib.animation import FuncAnimation, PillowWriter

plt.ioff()

# Create figure for on-fault evolution
fig1 = plt.figure(figsize=(10,10))
ax3 = fig1.add_subplot(3,1,1)
ax3.set_ylabel('Slip [m]')
ax3.set_ylim([0, 20])
ax3.set_xlim([0, Ly])

ax4 = fig1.add_subplot(3,1,2)
ax4.set_ylabel('Slip rate [m/s]')
ax4.set_ylim([0, 7])
ax4.set_xlim([0, Ly])

ax5 = fig1.add_subplot(3,1,3)
ax5.set_xlabel('fault [km]')
ax5.set_ylabel('stress [MPa]')
ax5.set_ylim([50, 90])
ax5.set_xlim([0, Ly])

plt.tight_layout()

y_fault = Y_l[-1,:]

# Initialize lines
line3, = ax3.plot([], [], 'g', lw=2, label='slip')
line4, = ax4.plot([], [], 'g', lw=2, label='slip rate')
line5, = ax5.plot([], [], 'g', lw=2, label='traction')
ax3.legend()
ax4.legend()
ax5.legend()

# Animation update function
def update_fault(frame_it):
    slip_ = FaultOutput[:, frame_it, 4]
    sliprate_ = np.sqrt(FaultOutput[:, frame_it, 0]**2 + FaultOutput[:, frame_it, 1]**2)
    traction_ = FaultOutput[:, frame_it, 3]
    
    line3.set_data(y_fault, slip_)
    line4.set_data(y_fault, sliprate_)
    line5.set_data(y_fault, traction_)
    
    fig1.suptitle(f'On-Fault Evolution - Time: {frame_it*dt:.2f} s')
    return line3, line4, line5

# Create animation (sample every 5th frame)
frames_fault = range(0, nt, 5)
anim_fault = FuncAnimation(fig1, update_fault, frames=frames_fault, blit=True, interval=50)

# Save animation as GIF
print("Saving on-fault evolution as GIF...")
writer = PillowWriter(fps=20)
anim_fault.save('fault_evolution.gif', writer=writer)
print("Saved: fault_evolution.gif")

# Save final state as PNG
update_fault(nt-1)
fig1.savefig('fault_evolution_final.png', dpi=300, bbox_inches='tight')
print("Saved: fault_evolution_final.png")

# Delete animation to free memory
del anim_fault

    

Saving on-fault evolution as GIF...
Saved: fault_evolution.gif
Saved: fault_evolution_final.png


In [25]:
#YT = np.zeros((ny, nt))
#TY = np.zeros((ny, nt))
VT = np.zeros((nt, ny))

for it in range(nt):
    for j in range(ny):
#        YT[j, it] = y_fault[j,0]
#        TY[j, it] = it*dt
        VT[it, j] = np.sqrt(FaultOutput[j, it, 0]**2 + FaultOutput[j, it, 1]**2)
        

#VT = np.sqrt(FaultOutput[:, :, 0]**2 + FaultOutput[:, :, 1]**2)        

fig_slip_time = plt.figure(figsize=(10, 6))
v = 2.5
image = plt.imshow(VT, aspect='auto', extent=[0, Ly, nt*dt, 0],
                   cmap='viridis', vmin=0, vmax=+v, interpolation='none')
        
plt.colorbar(label='Slip rate [m/s]')
plt.xlabel('fault [km]')
plt.ylabel('t [s]')
plt.title('Slip Rate Through Time')

# Save as PNG
fig_slip_time.savefig('slip_rate_spacetime.png', dpi=300, bbox_inches='tight')
print("Saved: slip_rate_spacetime.png")
plt.show()

Saved: slip_rate_spacetime.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>